In [139]:
%pylab inline
import pandas as pd
import seaborn as sns
from io import StringIO
import requests
import json
import plotly.express as px
import ipywidgets as widgets
from datetime import datetime
import imageio
import os

Populating the interactive namespace from numpy and matplotlib


/Users/annette/.local/share/virtualenvs/covid-8MZFs_Ml/lib/python3.8/site-packages/IPython/core/magics/pylab.py:159: UserWarning:

pylab import has clobbered these variables: ['title', 'datetime']
`%matplotlib` prevents importing * from pylab and numpy



In [140]:
def get_geojson_file(filename):
    with open('geojson/'+filename) as json_file:
        data = json.load(json_file)
        return data

#URL_province_geojson="https://raw.githubusercontent.com/openpolis/geojson-italy/master/geojson/limits_IT_provinces.geojson"
filename_municipalities = "limits_IT_municipalities.geojson"

def get_geojson_file(filename):
    with open('geojson/'+filename) as json_file:
        data = json.load(json_file)
        return data


geojson_province = get_geojson_file(filename_municipalities)
#geojson_province = getGeoJson(URL_province_geojson)

len([g['properties']['prov_acr'] for g in geojson_province['features']])

bz_features = [g for g in geojson_province['features'] if g['properties']['prov_acr']=='BZ']
bz_geojson = geojson_province['features'] = bz_features


# Get COVID data

In [144]:
tested_link = "http://www.provinz.bz.it/news/de/news.asp?news_action=300&news_image_id=1061992"
qurantaene_link = "http://www.provinz.bz.it/news/de/news.asp?news_action=300&news_image_id=1061998"

def get_excel_url(url, header=0):
    s=requests.get(url)
    #print(s.content)
    return pd.read_excel(s.content, header=header)
  
# get quarantined
quaran = get_excel_url(qurantaene_link, header=2)

#get test positive and clean
tested = get_excel_url(tested_link, header=2)
tested = tested[tested['Codice Istat Comune'].notna()] # remove lines with na
tested = tested[tested['Struttura di provenienza richiesta'].isna()] # only take total

# put together
tested = pd.DataFrame({
    "Cod Istat": tested['Codice Istat Comune'].astype(int),
    "positives": tested['Totali al 26-03-2020'].astype(int),
})
# tested.set_index(['Cod Istat'], inplace=True)
#quaran.set_index(['Cod Istat'], inplace=True)
quaran = quaran.rename(columns={"Numero casi al 26-03-2025": "quarantined"})

df = pd.concat([quaran, tested]).groupby('Cod Istat').last().fillna(value=0)

df = df.drop(index = "Totale complessivo") # remove total row
#df = df.drop(index = "21008") # remove total row

df = df.reset_index()

total_quarantined = df["quarantined"].sum()
total_positves = df["positives"].sum()


# plot map

In [161]:

def bz_map(file, title, color_field = "quarantined", max_value=None):
    if not max_value:
        max_value = file[color_field].max()
    
    total_quarantined = file["quarantined"].sum()
    total_positves = file["positives"].sum()
    
    total_quarantined = 3215
    total_positves = 910
    
    number = "Total Quarantined: %s, COVID Positive: %s" % (int(total_quarantined), int(total_positves))
    #print(number)
    fig = px.choropleth(file, geojson=geojson_province, 
                        color=color_field,
                        range_color=(0, max_value),
                        color_continuous_scale=["white", "Red"],
                        locations="Cod Istat", 
                        featureidkey="properties.com_istat_code_num",
                        projection="mercator",
                        hover_data=["Wohngemeinde", "positives", "quarantined"],
                        labels={color_field:'Total '+color_field}
                       )
    fig.update_geos(fitbounds="locations", visible=False)
    fig.update_layout(
        margin={"r":0,"t":30,"l":0,"b":0},
        title={
        'text': title + number,
        'y':0.93,
        'x':0.1},
        title_font_size=20,
        
        #geo_scope='europe', # limite map scope to Europe
        annotations = [dict(
            x=0.55,
            y=0.02,
            xref='paper',
            yref='paper',
            text='Source: <a href="http://www.provinz.bz.it/>http://www.provinz.bz.it</a><br>©Copyright Annette Werth',
            showarrow = False
    )]
   )
    return fig

title = "26. März: COVID-positive Südtiroler<br>"
fig = bz_map(df, title, color_field="positives" )
fig.show()
